In [14]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd

# 6. Gesell*

Adapted from [Unit 9: Gesell.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/Gesell.odc).

Gesell Developmental Schedules measure child development. The scoring is devised by the American child psychologist and pediatrician Arnold Gesell who founded the Clinic of Child Development at Yale in 1911 and directed it for many years. The Gesell Developmental Schedules are a gauge of the status of a child's motor and language development and personal-social and adaptive behaviors.

For 21 children the age in months at which they first spoke and their Gesell Adaptive Score, which is the result of an aptitude test taken much later was recorded. These data were originally collected by L. M. Linde of UCLA but were first published by M. R. Mickey, O. J. Dunn, and V. Clark, "Note on the use of stepwise regression in detecting outliers," *Computers and Biomedical Research*, 1 (1967), pp.105-111.

The data have been used by several authors, e.g., N. R. Draper and J. A. John, "Influential observations and outliers in regression,"
*Technometrics*, 23 (1981), pp. 21-26.

Let $X$ be the age in months a child speaks his/her first word and let $Y$ be the Gesell adaptive score, a measure of a child's aptitude (observed later on):

| Child  | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 |
|--------|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|
| Age    | 15 | 26 | 10 | 9  | 15 | 20 | 18 | 11 | 8  | 20 | 7  | 9  | 10 | 11 | 11 | 10 | 12 | 42 | 17 | 11 | 10 |
| Score  | 95 | 71 | 83 | 91 | 102| 87 | 93 |100 |104 | 94 |113 | 96 | 83 | 84 |102 |100 |105 | 57 |121 | 86 |100 |



A correlation of $r$ = 0.640 indicates a  moderately strong negative
relationship between age at first word spoken and Gesell score.

How does the child's aptitude change with how long it takes them to speak?

Are there any influential observations/potential outliers?

The solution follows a two stage modeling.

 - a.) Obtain mean ICPOs then take reciprocal of mean ICPOs (these are CPOs).  and consder them as an input for model B

 - b.) Read in and calibrate CPOs with posterior ordinates.

In [15]:
data = pd.read_csv("../data/gesell.csv")
y = data["Score"].to_numpy()
x = data["Age"].to_numpy()

## Part A

In [16]:
with pm.Model() as model_a:
    # couldn't get .001 working, answers are pretty close with .003
    tau = pm.Gamma("tau", 1, 0.003)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.00001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "cpo", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    # what is the point of this when we want cpo anyways?
    icpo = pm.Deterministic("icpo", 1 / f)
    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    trace = pm.sample(100000, target_accept=0.99)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


Output()

Sampling 4 chains for 1_000 tune and 100_000 draw iterations (4_000 + 400_000 draws total) took 63 seconds.


In [17]:
icpo_df = az.summary(trace, var_names="icpo", round_to=5, filter_vars="like")

ICPO results match BUGS.

In [18]:
icpo_df

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
icpo[0],28.47712,4.76697,20.42687,37.55074,0.01255,0.01179,152195.58152,177950.94488,1.00002
icpo[1],48.01467,25.44420,22.41644,86.64288,0.05717,0.28733,212052.38152,252270.97701,1.00001
icpo[2],92.80916,50.33783,36.48155,174.61153,0.11002,0.27689,193461.86560,230507.07337,1.00001
icpo[3],40.57548,10.57165,24.96387,59.74110,0.02515,0.03486,188027.21518,223579.06862,1.00002
icpo[4],40.82310,8.74666,26.82649,56.81026,0.01558,0.02447,351813.73329,274187.15269,1.00002
icpo[5],28.32800,4.89525,19.98275,37.46224,0.01326,0.01295,144134.63455,168488.68824,1.00002
icpo[6],29.64629,5.16439,20.86118,39.28246,0.01300,0.01239,167301.35230,198545.03695,1.00003
icpo[7],28.91433,4.94631,20.53986,38.24434,0.01311,0.01246,150663.85761,175057.72284,1.00003
icpo[8],29.79322,5.55492,20.59258,40.03976,0.01510,0.01646,146258.06141,167295.89775,1.00003
icpo[9],34.93048,7.73598,22.79149,49.03943,0.01644,0.02260,236893.91662,249085.30812,1.00002


In [19]:
cpo_means = 1 / icpo_df["mean"].to_numpy()
cpo_means

array([0.03511591, 0.02082697, 0.0107748 , 0.02464543, 0.02449593,
       0.03530076, 0.03373103, 0.03458493, 0.03356468, 0.02862829,
       0.019232  , 0.03314277, 0.0107748 , 0.01492528, 0.03239641,
       0.0351815 , 0.02518653, 0.01454344, 0.00015413, 0.01910296,
       0.0351815 ])

## Part B

In [20]:
mlcpo = -2 * np.log(cpo_means)
MPQ = np.sum(mlcpo)

In [21]:
with pm.Model() as model_b:
    tau = pm.Gamma("tau", 1, 0.002)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.00001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "f", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    c = pm.Deterministic("c", cpo_means / f)

    g1 = pm.Deterministic("g1", c * pm.math.log(c))
    g2 = -pm.math.log(c)
    g3 = 0.5 * np.absolute((c - 1))
    g4 = (c - 1) ** 2

    pm.Deterministic("s_g1", pm.math.sum(g1))
    pm.Deterministic("s_g2", pm.math.sum(g2))
    pm.Deterministic("s_g3", pm.math.sum(g3))
    pm.Deterministic("s_g4", pm.math.sum(g4))

    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    traceb = pm.sample(
        2000, target_accept=0.95, idata_kwargs=dict(log_likelihood=True)
    )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 1 seconds.


In [22]:
az.summary(traceb, var_names="c", filter_vars="like", kind="stats")

,mean,sd,hdi_3%,hdi_97%
c[0],0.998,0.165,0.715,1.309
c[1],1.011,0.508,0.462,1.799
c[2],0.990,0.517,0.395,1.829
c[3],0.995,0.257,0.618,1.455
c[4],0.995,0.209,0.660,1.386
c[5],0.999,0.170,0.710,1.314
c[6],0.997,0.173,0.722,1.339
c[7],0.998,0.168,0.707,1.309
c[8],0.998,0.182,0.694,1.332
c[9],0.993,0.218,0.661,1.405


In [23]:
az.summary(traceb, var_names="s_g", filter_vars="like", kind="stats")

,mean,sd,hdi_3%,hdi_97%
s_g1,3.335,84.472,-2.737,8.271
s_g2,2.457,1.209,0.242,3.913
s_g3,2.805,6.497,1.146,5.082
s_g4,163.179,12291.836,0.599,13.229


In [24]:
az.summary(traceb, var_names="g1", kind="stats")

,mean,sd,hdi_3%,hdi_97%
g1[0],0.011,0.176,-0.263,0.326
g1[1],0.107,0.797,-0.361,1.040
g1[2],0.095,0.777,-0.368,1.077
g1[3],0.025,0.306,-0.314,0.522
g1[4],0.016,0.232,-0.307,0.410
g1[5],0.013,0.183,-0.253,0.342
g1[6],0.011,0.184,-0.252,0.360
g1[7],0.012,0.179,-0.253,0.343
g1[8],0.014,0.197,-0.255,0.381
g1[9],0.015,0.245,-0.292,0.441


In [25]:
az.waic(traceb)

/Users/rob/anaconda3/envs/ISYE_6420_Bayes_PyMC/lib/python3.11/site-packages/arviz/stats/stats.py:1655: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 8000 posterior samples and 21 observations log-likelihood matrix.

          Estimate       SE
elpd_waic   -82.54     4.98
p_waic        3.25        -

There has been a warning during the calculation. Please check the results.

In [27]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: Sat Jul 19 2025

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.17.2

pytensor: 2.30.3

pymc  : 5.22.0
pandas: 2.1.3
arviz : 0.21.0
numpy : 1.25.2

